In [ ]:
import os.path, json
from syrocr.images import Im, AvgIm
# https://stackoverflow.com/a/32370538
# usage: display(image)
from IPython.core.display import display

In [ ]:
basename = 'vts-088_2L'
img_dir = '../analyzevts/scans/genesis-exodus/exodus'
json_dir = '../analyzevts/scans/genesis-exodus/exodus/test'
source_image = os.path.join(img_dir, basename + '.tif')
lines_file = os.path.join(json_dir, basename + '_lines.json')
text_file = os.path.join(json_dir, basename + '_textlines.json')
tables_file = os.path.join(json_dir, 'genesis_tables2.json')

In [ ]:
with open(lines_file, 'r') as f:
    lines = json.load(f)

with open(text_file, 'r') as f:
    text = json.load(f)

with open(tables_file, 'r') as f:
    tables = json.load(f)

im = Im(source_image)

In [ ]:
# TODO textsize should be available in line dict
def get_textsize(linetype, section):
    if section == 'main' and linetype in ('text', 'pagenr'):
        textsize = 'normal'
    else:
        textsize = 'small'
    return textsize

# we are mainly interested in the 'main' section,
# so for now we ignore 'marginl' and 'marginr'
section = 'main'

In [ ]:
for line, textline in zip(lines[1:], text[1:]):
    linetype = line['type']
    textsize = get_textsize(linetype, section)
    table = tables[textsize]
    display(im.crop(line['main']))
    print('Line', line['num'], 'textsize', textsize)
    for i, (c_id, d, keyoverride, box) in enumerate(textline[section]):
        c = table[c_id]

        print(i, c_id, d, keyoverride, c['key'])

        if c['key'] is None:

            print(f'page {basename} line {line["num"]} character {i} id {c_id}')

            avgim = AvgIm(c['avgim']['base64_str'], c['avgim']['baseline'], c['avgim']['width'], c['avgim']['height'])
            display(avgim.maxtoblack(invert=True))

            key = input().split()
            
            table[c_id]['key'] = {
                'tr':     '' if not key else key.pop(0),
                'script': '' if not key or key[0].isdigit() else key.pop(0),
                'dist':   None if not key else int(key.pop(0)),
            }
            
            print(table[c_id]['key'])

In [ ]:
# dialogue to prevent accidental writing to tables file
if input(f'Type \'yes indeed\' to save the tables dict to {tables_file}:\n') == 'yes indeed':
    with open(tables_file, 'w') as f:
        json.dump(tables, f)
    print('Saved!')
else:
    print('Not saved.')